In [ ]:
from langchain.chat_models import ChatOpenAI
from utils.env_loader import get_env
api_key = get_env("ZHIPUAI_API_KEY")

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


# 创建CharModels
# ChatModels输入是一个ChatMEssage列表
# ChatModels输出是一个单独的ChatMessage
# ChatMessage的必要组件：content和role（由ChatMessage来定义的实体对象）
# Langchain提供HumanMessage，AIMessage，SystemMessage，FunctionMessage，可以通过ChatMessage自定义角色

# 这是使用ZhipuAI的标准模板，
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)


In [ ]:
# predict_message方法对消息列表处理，返回的也是消息类型
from langchain.schema import HumanMessage

text = "制造多彩袜子的公司有什么好名字？"
message = [HumanMessage(content=text)]

llm.predict_messages(message)

/tmp/ipykernel_37643/323288062.py:6: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict_messages(message)


AIMessage(content='制造多彩袜子的公司可以考虑以下一些有创意的名字，这些名字旨在体现出产品丰富多彩的特点，同时也要易于记忆和传播：\n\n1. 彩纬编织\n2. 缤纷足语\n3. 彩绘袜界\n4. 色彩踏步\n5. 梦幻袜彩\n6. 色彩漫步\n7. 彩虹桥袜业\n8. 色彩足迹\n9. 风彩袜艺\n10. 彩韵袜品\n\n选择名字时，还应该考虑以下因素：\n- 名字是否已被注册或商标保护，避免侵权。\n- 名字是否易于在国内外市场发音和传播。\n- 名字是否能够体现出公司的文化理念和产品特色。\n\n希望这些建议能够帮助您找到一个既响亮又吸引人的公司名字！', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 176, 'prompt_tokens': 14, 'total_tokens': 190, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-74b90b0a-cf2d-4397-a836-8df9d73796bd-0')

PromptTemplates

In [ ]:
from langchain.prompts import PromptTemplate
# 这里可以看到使用模板函数时指定的参数用{}标出，参数为其中的内容
prompt = PromptTemplate.from_template("制造{product}的公司有什么好名字？")
prompt.format(product="玩具")


'制造玩具的公司有什么好名字？'

In [20]:
from langchain.prompts.chat import(
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
# 显式定义模板
system_template = "你是一个强力的翻译助手，可以将{input_language}，翻译为{output_language}."
sys_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([sys_message_prompt, human_message_prompt])
# 这里定义模板参数
messages = chat_prompt.format_messages(input_language="英语",output_language="中文",text="i Love programming.")


In [24]:
chat_prompt

ChatPromptTemplate(input_variables=['input_language', 'output_language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], input_types={}, partial_variables={}, template='你是一个强力的翻译助手，可以将{input_language}，翻译为{output_language}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [23]:
messages

[SystemMessage(content='你是一个强力的翻译助手，可以将英语，翻译为中文.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i Love programming.', additional_kwargs={}, response_metadata={})]

In [21]:
llm.predict_messages(messages)

AIMessage(content="我喜欢编程。 \n\nIf you need any translation help, feel free to ask. If you have any English text you'd like to translate into Chinese, just provide it and I'll do my best to translate it for you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 25, 'total_tokens': 72, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1a875ea3-9040-4d16-93ab-8199c55867c0-0')

# 输出解析器

In [22]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split("，")

CommaSeparatedListOutputParser().parse("hi，bye")

['hi', 'bye']

# LLMChian

In [28]:
# 将多种组件组合到一起形成一个链式组件
from langchain.chains import LLMChain

chain = LLMChain(
    llm = llm,
    prompt = chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)

In [34]:
# 在运行时补全所有的参数
chain.run(input_language="英文",output_language="中文", text="i love programming.")

["我喜欢编程。 \n\nIf you need any translation help, feel free to ask me. If you have any specific sentences or texts you'd like to translate, just let me know!"]

# LCEL Langchain Expression Language
构建组合链式操作chains的声明语言

以下是这个过程的说明示例

In [ ]:
from langchain_core.output_parsers import StrOutputParser
# 这一过程可以参考LLMchain
# 也即这个链式可以表示为 prompt | model | output_parser

# 可以表示为
chain = LLMChain(
    # PromptTemplate 类型
    # ChatModel类型
    # OutputParser类型
)
# 注意prompt的参数可以在调用chain前就指明也可以调用时再指明
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
# 指明参数
prompt_valued = prompt.invoke({"topic": "ice cream"})

# 这里当langchian没有默认支持时
# 需要查看各模型SDK的支持方案
model = ChatOpenAI(
    # 模型参数
)
# 此时可以接收指定参数后的prompt
message = model.invoke(prompt_valued)

output_parser = StrOutputParser()
# 此时可以直接解析message
output = output_parser(message)

# 也可以表示为
chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})


# 一个RAG简单过程

In [17]:
# 检索向量数据库
# 生成prompt
# 将prompt输入到模型
# 处理模型返回数据
from utils.env_loader import get_env
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
# 这里若需要多种不同的模板则可导入更多PromptTemplate类型
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from embedding.zhipu_embedding import ZhipuAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["ben worked at shanghai", "kety likes playing ball"],
    embedding=ZhipuAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

prompt_template = """Answer the question based only on the following context:
{context}

Question: {question}

"""
api_key = get_env("ZHIPUAI_API_KEY")
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

output_parser = StrOutputParser()


# 这里将retriever和RunnablePassthrough拼在一起
# 这里retriever会根据RunnablePassthrough传递的prompt['question']内容检索
# 然后生成一个检索的内容
# 最终setup_and_retriever返回
# {"context":, "question":}
setup_and_retriever = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retriever | prompt | model | output_parser

chain.invoke("where did ben work?")


'Ben worked at Shanghai.'

In [7]:
retriever.invoke("where did ben work?")

[Document(page_content='ben worked at shanghai'),
 Document(page_content='kety likes playing ball')]

In [8]:
RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

{
  context: VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7fb0da8f69c0>),
  question: RunnablePassthrough()
}

In [ ]:
# 说明RunnableParallel在做什么
# 他接收参数
# 将retriever的内容放在键为"context"下
# 将传入内容通过Runnablepassthrough()放在键question下

"""
下面这个组件放在链里会直接将参数传给prompt
类似于
setup_and_retriever | prompt
变为了
prompt({'context':retriever, 'question':'where is ben work'})
""" 

setup_and_retriever.invoke("where is ben work")

{'context': [Document(page_content='ben worked at shanghai'),
  Document(page_content='kety likes playing ball')],
 'question': 'where is ben work'}

In [10]:
prompt.invoke({"context": "test", "question": "test"})

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\ntest\n\nQuestion: test\n\n')])

# 一个流式传输的例子

流式传输是为了更好的与用户交互，如果不使用流式传输，用户将等待很长时间直接看到一长串内容


In [13]:
chunks = []

async for chunk in model.astream("你好告诉我一些关于你的事情"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

你好|！|我是一个|人工智能|助手|，|基于|大型|语言|模型|开发|而成|，|我的|任务是|针对|用户|的问题|和要求|提供|适当的|答复|和支持|。|我可以|处理|多种|语言|任务|，|包括|文本|生成|、|翻译|、|信息|检索|等|。|我|由|清华大学| K|EG| 实|验|室|与|智|谱|AI|共同|训练|，|我的|目标|是通过|与|人类的|互动|，|帮助|人们|获取|信息|、|解决问题|，|让|生活|变得更加|便捷|。|我|遵循|中国|政府的|立场|，|坚守|社会主义|价值观|。|在与|你|互动|的过程中|，|我会|不断|学习和|进步|，|以期|提供|更|优质|的服务|。||

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("告诉我一个关于{topic}的笑话")

# 从输出中可以发现，这个解析器会将模型的输出进行换行解析
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic" : "鹦鹉"}):
    print(chunk, end="|", flush=True)

好的|，|这里|有一个|关于|鹦|鹉|的|笑话|：

有一天|，|一个人|去|宠物|店|买|了一只|鹦|鹉|。|他|问|鹦|鹉|：“|你会|说话|吗|？”|鹦|鹉|回答|：“|当然|会|，|我|可是|只|聪明的|鹦|鹉|。”

回家|后|，|主人|发现|这只|鹦|鹉|确实|很|聪明|，|它能|模仿|各种|声音|，|甚至|能|学说|一些|简单的|词语|。|有一天|，|主|人在|厨房|做饭|，|鹦|鹉|在|客厅|里|玩|。|突然|，|主人|听到|鹦|鹉|在|尖叫|：“|救命|啊|！|救命|啊|！|”

主人|急忙|跑|出|厨房|，|发现|鹦|鹉|正|吊|在|窗帘|上|，|脚|被|窗帘|绳|缠|住了|。|主人|赶紧|把|鹦|鹉|解|救|下来|，|问|它|：“|你|刚才|为什么|喊|救命|？|”

鹦|鹉|答|道|：“|我只是|想|试试|，|看|你会|不会|真的|来|救|我|。”

主人|哭|笑|不得|，|心想|：|原来|这只|鹦|鹉|不仅|聪明|，|还|这么|调皮|。||

In [21]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "......."
         ),
        ("human","........")
    ]
)

In [22]:
print(prompt)

input_variables=[] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='.......')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='........'))]


# RunnablePassthrough:传递参数

# ChatOpenAI().bind():绑定运行时参数
如何时停止，对于模型特定的运行脚本

